In [7]:
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from torch.utils.data import DataLoader, Subset
import random
import numpy as np
from tqdm import tqdm
import soundfile as sf

In [12]:
from torchaudio.datasets import SPEECHCOMMANDS

class SubsetSC(SPEECHCOMMANDS):
    def __init__(self, subset):
        super().__init__("./", download=True)

        def load_list(filename):
            filepath = os.path.join(self._path, filename)
            with open(filepath) as f:
                return [os.path.join(self._path, line.strip()) for line in f]

        if subset == "validation":
            self._walker = load_list("validation_list.txt")
        elif subset == "testing":
            self._walker = load_list("testing_list.txt")
        elif subset == "training":
            excludes = load_list("validation_list.txt") + load_list("testing_list.txt")
            excludes = set(excludes)
            self._walker = [w for w in self._walker if w not in excludes]

    def __getitem__(self, n):
        file_path = self._walker[n]
        # Load using soundfile instead of torchaudio.load
        waveform, sample_rate = sf.read(file_path, dtype="float32")
        waveform = torch.tensor(waveform).unsqueeze(0)  # (1, N)
        label = file_path.split(os.sep)[-2]
        return waveform, sample_rate, label, file_path

# Instantiate splits
train_data = SubsetSC("training")
test_data = SubsetSC("testing")

# Label mapping
labels = sorted(set(datapoint[2] for datapoint in train_data))
label_to_idx = {label: i for i, label in enumerate(labels)}

# Collate function
def collate_fn(batch):
    tensors, targets = [], []
    max_len = 16000  # 1 second at 16kHz

    for waveform, _, label, *_ in batch:
        if waveform.size(1) < max_len:
            padding = max_len - waveform.size(1)
            waveform = F.pad(waveform, (0, padding))
        else:
            waveform = waveform[:, :max_len]  # Trim if too long
        tensors.append(waveform)
        targets.append(label_to_idx[label])

    tensors = torch.stack(tensors)
    targets = torch.tensor(targets)
    return tensors, targets
# Dataloaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False, collate_fn=collate_fn)


In [13]:
class AudioCNN(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 8, kernel_size=9, stride=1)
        self.pool = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(8, 16, kernel_size=9, stride=1)

        # Dummy input to calculate flatten dimension dynamically
        self._dummy_input = torch.zeros(1, 1, 16000)
        self.flatten_dim = self._get_flatten_dim()

        self.fc1 = nn.Linear(self.flatten_dim, 64)
        self.fc2 = nn.Linear(64, n_classes)

    def _get_flatten_dim(self):
        x = self.pool(F.relu(self.conv1(self._dummy_input)))
        x = self.pool(F.relu(self.conv2(x)))
        return x.view(1, -1).size(1)

    def forward(self, x):
        x = x[:, 0:1, :]  # Ensure mono
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# Instantiate the model here
model = AudioCNN(n_classes=len(labels))
print(f"Total trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")


Total trainable parameters: 1024515


In [14]:
def rho_loss(outputs, targets, model_confidence_threshold=0.5):
    probs = F.softmax(outputs, dim=1)
    max_probs, _ = torch.max(probs, dim=1)
    weights = (max_probs < model_confidence_threshold).float()
    ce_loss = F.cross_entropy(outputs, targets, reduction='none')
    return (ce_loss * weights).mean()


In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    batch_losses = []

    for batch_idx, (x, y) in enumerate(tqdm(train_loader)):
        x, y = x.to(device), y.to(device)
        outputs = model(x)
        loss = rho_loss(outputs, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        batch_losses.append(loss.item())

        print(f"Epoch [{epoch+1}/{num_epochs}] Batch [{batch_idx+1}/{len(train_loader)}] Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"\n>> Epoch {epoch+1} Average Loss: {avg_loss:.4f}\n")


  0%|          | 5/1654 [00:02<08:42,  3.16it/s]

Epoch [1/5] Batch [1/1654] Loss: 3.5644
Epoch [1/5] Batch [2/1654] Loss: 3.6305
Epoch [1/5] Batch [3/1654] Loss: 3.5738
Epoch [1/5] Batch [4/1654] Loss: 3.5039
Epoch [1/5] Batch [5/1654] Loss: 3.5567
Epoch [1/5] Batch [6/1654] Loss: 3.4983
Epoch [1/5] Batch [7/1654] Loss: 3.5787
Epoch [1/5] Batch [8/1654] Loss: 3.5608


  1%|          | 13/1654 [00:02<02:45,  9.92it/s]

Epoch [1/5] Batch [9/1654] Loss: 3.5518
Epoch [1/5] Batch [10/1654] Loss: 3.5400
Epoch [1/5] Batch [11/1654] Loss: 3.5099
Epoch [1/5] Batch [12/1654] Loss: 3.5133
Epoch [1/5] Batch [13/1654] Loss: 3.5932
Epoch [1/5] Batch [14/1654] Loss: 3.5294
Epoch [1/5] Batch [15/1654] Loss: 3.5581
Epoch [1/5] Batch [16/1654] Loss: 3.5949


  1%|▏         | 21/1654 [00:02<01:32, 17.62it/s]

...

Epoch [5/5] Batch [1637/1654] Loss: 1.0954
Epoch [5/5] Batch [1638/1654] Loss: 1.5451
Epoch [5/5] Batch [1639/1654] Loss: 1.6151
Epoch [5/5] Batch [1640/1654] Loss: 1.2437
Epoch [5/5] Batch [1641/1654] Loss: 1.0398
Epoch [5/5] Batch [1642/1654] Loss: 1.0367
Epoch [5/5] Batch [1643/1654] Loss: 1.2700


100%|█████████▉| 1648/1654 [00:50<00:00, 33.76it/s]

Epoch [5/5] Batch [1644/1654] Loss: 1.4678
Epoch [5/5] Batch [1645/1654] Loss: 1.2155
Epoch [5/5] Batch [1646/1654] Loss: 1.1028
Epoch [5/5] Batch [1647/1654] Loss: 1.3107
Epoch [5/5] Batch [1648/1654] Loss: 1.2125
Epoch [5/5] Batch [1649/1654] Loss: 1.3568
Epoch [5/5] Batch [1650/1654] Loss: 1.1844
Epoch [5/5] Batch [1651/1654] Loss: 1.3939


100%|██████████| 1654/1654 [00:50<00:00, 32.95it/s]

Epoch [5/5] Batch [1652/1654] Loss: 1.2347
Epoch [5/5] Batch [1653/1654] Loss: 1.2352
Epoch [5/5] Batch [1654/1654] Loss: 1.6821

>> Epoch 5 Average Loss: 1.3069



In [16]:
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    print(f"Accuracy: {correct / total * 100:.2f}%")

evaluate(model, test_loader)


Accuracy: 51.00%
